### TESTING NEW FUNCTIONS

In [1]:
import pandas as pd
import getout_of_text_3 as got3
from getout_of_text_3 import ScotusAnalysisTool, ScotusFilteredAnalysisTool
from langchain.chat_models import init_chat_model

# print version of getout_of_text_3
print('Running getout_of_text3 version:', got3.__version__)


Running getout_of_text3 version: 0.3.3


In [60]:
def export_markdown_reasoning_first(result, keyword: str, filename: str | None = None):
    """Export a markdown report with REASONING CONTENT first, then the rest.

    Ordering Rules:
    1. # reasoning content  (aggregated reasoning_content list or string)
    2. Remaining known sections in this order if present: summary, occurrences_summary, limitations, total_contexts.
    3. Any other string fields in the dict are appended at the end under a generic heading.

    Parameters
    ----------
    result : dict | str | list
        Output from the SCOTUS analysis tool (JSON mode recommended for richer structure).
    keyword : str
        Used to build filename (sanitized) if filename not provided.
    filename : str | None
        Optional explicit filename. If None, auto-generated.
    """
    import json, re, os

    def _sanitize(name: str) -> str:
        cleaned = ''.join(c if (c.isalnum() or c in ('-','_')) else '_' for c in name.strip())
        return cleaned or 'analysis'

    reasoning_block = ''
    body_sections = []

    if isinstance(result, dict):
        rc = result.get('reasoning_content')
        if isinstance(rc, list):
            reasoning_block = '\n'.join(str(x) for x in rc if str(x).strip())
        elif isinstance(rc, str):
            reasoning_block = rc.strip()
        else:
            # try nested style
            if isinstance(rc, dict) and 'text' in rc:
                reasoning_block = str(rc['text']).strip()
        # Collect ordered sections
        ordered_keys = ['summary', 'occurrences_summary', 'limitations', 'total_contexts']
        used = set(['reasoning_content'])
        for k in ordered_keys:
            if k in result and k not in used:
                val = result[k]
                if isinstance(val, (str, int, float)):
                    body_sections.append(f"## {k}\n\n{val}\n")
                else:
                    body_sections.append(f"## {k}\n\n{json.dumps(val, indent=2)}\n")
                used.add(k)
        # Append any other simple string fields not yet used
        for k, v in result.items():
            if k in used:
                continue
            if isinstance(v, str) and v.strip():
                body_sections.append(f"## {k}\n\n{v.strip()}\n")
            elif isinstance(v, (int, float)):
                body_sections.append(f"## {k}\n\n{v}\n")
    elif isinstance(result, list):
        # Try to extract reasoning blocks and remainder
        reasoning_parts = []
        other_parts = []
        for block in result:
            if isinstance(block, dict):
                rc = block.get('reasoning_content')
                if isinstance(rc, list):
                    reasoning_parts.extend(str(x) for x in rc if str(x).strip())
                elif isinstance(rc, str) and rc.strip():
                    reasoning_parts.append(rc.strip())
                if 'text' in block and isinstance(block['text'], str):
                    other_parts.append(block['text'])
            elif isinstance(block, str):
                other_parts.append(block)
        reasoning_block = '\n'.join(reasoning_parts)
        if other_parts:
            body_sections.append("## response\n\n" + '\n\n'.join(other_parts))
    else:  # plain string
        reasoning_block = ''
        body_sections.append(f"## response\n\n{str(result)}")

    # Fallback if no reasoning extracted
    if not reasoning_block:
        reasoning_block = '(No explicit reasoning_content found in result)'

    report = ["# reasoning content\n", "```text\n", reasoning_block, "\n```\n\n"]
    report.extend(body_sections)

    safe_keyword = _sanitize(keyword)
    outname = filename or f"{safe_keyword}_reasoning_first.md"
    with open(outname, 'w', encoding='utf-8') as f:
        f.write('\n'.join(report))
    print(f"Markdown report written: {outname} (length={sum(len(x) for x in report)} chars)")
    return outname

# Example export for the filtered JSON analysis (only if 'analysis' exists)
try:
    if 'analysis' in globals():
        export_markdown_reasoning_first(analysis, 'ordinary_meaning')
except Exception as e:
    print(f"Export failed: {e}")

# Utility: export a LangChain / Bedrock style streamed block list (reasoning + text) to markdown
from datetime import datetime
from pathlib import Path

def export_blocks_reasoning_first(blocks, keyword: str, filename: str | None = None):
    """Export a list of model blocks (each a dict with 'type' and maybe 'reasoning_content' or 'text')
    to a markdown report where the reasoning content appears FIRST.

    Expected block shapes (any others are stringified):
      {'type': 'reasoning_content', 'reasoning_content': {'text': '...'}}
      {'type': 'text', 'text': '...'}

    Ordering:
      1. # reasoning content (aggregate all reasoning text blocks in order)
      2. ## response (concatenate all text blocks)

    Each section fenced appropriately for readability. Returns output filepath.
    """
    def _sanitize(name: str) -> str:
        return ''.join(c if (c.isalnum() or c in ('-','_')) else '_' for c in name.strip()) or 'analysis'

    reasoning_parts = []
    text_parts = []
    for b in blocks:
        if not isinstance(b, dict):
            text_parts.append(str(b))
            continue
        b_type = b.get('type')
        if b_type == 'reasoning_content':
            rc = b.get('reasoning_content')
            if isinstance(rc, dict) and 'text' in rc and rc['text']:
                reasoning_parts.append(str(rc['text']))
            elif isinstance(rc, str):
                reasoning_parts.append(rc)
        elif b_type == 'text':
            t = b.get('text')
            if isinstance(t, str):
                text_parts.append(t)
        else:  # fallback
            # include unknown block types in response section for transparency
            text_parts.append(str(b))

    reasoning_block = '\n\n'.join(p.strip() for p in reasoning_parts if p and p.strip())
    if not reasoning_block:
        reasoning_block = '(No reasoning_content blocks found)'
    response_block = '\n\n'.join(p.strip() for p in text_parts if p and p.strip()) or '(No text blocks found)'

    safe_keyword = _sanitize(keyword)
    outname = filename or f"{safe_keyword}_reasoning_first_stream.md"

    lines = []
    lines.append(f"# reasoning content\n")
    lines.append("```text\n")
    lines.append(reasoning_block)
    lines.append("\n```\n\n")
    lines.append("## response\n\n")
    lines.append(response_block)
    lines.append("\n\n---\n")
    lines.append(f"_generated: {datetime.utcnow().isoformat()}Z | keyword: {keyword}_")

    with open(outname, 'w', encoding='utf-8') as f:
        f.write(''.join(lines))
    print(f"Markdown (reasoning first) written: {outname} | length={sum(len(x) for x in lines)} chars")
    return outname

def normalize_and_export_filtered_analysis(filtered_result: dict, keyword: str, filename: str | None = None):
    """
    Specialized function to handle ScotusFilteredAnalysisTool output that often contains
    embedded JSON within string fields (especially 'summary').
    
    This function:
    1. Detects and extracts embedded JSON from string fields
    2. Merges the extracted structured data with top-level fields
    3. Exports to markdown with reasoning content first
    
    Parameters
    ----------
    filtered_result : dict
        Raw output from ScotusFilteredAnalysisTool._run()
    keyword : str
        Keyword used for filename generation
    filename : str | None
        Optional explicit filename
        
    Returns
    -------
    str
        Path to the generated markdown file
    """
    import json
    import re
    from datetime import datetime
    
    def _find_embedded_json(text: str) -> str | None:
        """Find JSON object in text using multiple patterns."""
        # Look for various JSON starting patterns
        patterns = [
            r'\{\s*"keyword":\s*"[^"]*"',  # {"keyword": "..."}
            r'\{\n\s*"keyword":\s*"[^"]*"',  # multiline variant
        ]
        
        for pattern in patterns:
            match = re.search(pattern, text)
            if match:
                start_pos = match.start()
                # Find matching closing brace
                depth = 0
                in_string = False
                escape = False
                
                for i, char in enumerate(text[start_pos:], start=start_pos):
                    if in_string:
                        if escape:
                            escape = False
                        elif char == '\\':
                            escape = True
                        elif char == '"':
                            in_string = False
                    else:
                        if char == '"':
                            in_string = True
                        elif char == '{':
                            depth += 1
                        elif char == '}':
                            depth -= 1
                            if depth == 0:
                                return text[start_pos:i+1]
        return None
    
    def _sanitize_filename(name: str) -> str:
        """Create safe filename from keyword."""
        return ''.join(c if (c.isalnum() or c in ('-','_')) else '_' for c in name.strip()) or 'analysis'
    
    # Step 1: Create working copy
    normalized = dict(filtered_result)
    extraction_notes = []
    
    # Step 2: Process each string field for embedded JSON
    for field_name, field_value in list(filtered_result.items()):
        if not isinstance(field_value, str) or '"keyword"' not in field_value:
            continue
            
        # Try to extract JSON
        json_text = _find_embedded_json(field_value)
        if not json_text:
            continue
            
        try:
            extracted_data = json.loads(json_text)
            extraction_notes.append(f"Extracted JSON from '{field_name}'")
            
            # Merge extracted data into normalized result
            for key, value in extracted_data.items():
                if key == 'reasoning_content':
                    # Special handling for reasoning_content - merge lists
                    existing = normalized.get('reasoning_content', [])
                    if isinstance(existing, str):
                        existing = [existing] if existing.strip() else []
                    elif not isinstance(existing, list):
                        existing = []
                    
                    new_items = []
                    if isinstance(value, list):
                        new_items = [str(item).strip() for item in value if str(item).strip()]
                    elif isinstance(value, str) and value.strip():
                        new_items = [value.strip()]
                    
                    # Combine and deduplicate while preserving order
                    combined = existing + new_items
                    seen = set()
                    deduplicated = []
                    for item in combined:
                        if item not in seen:
                            seen.add(item)
                            deduplicated.append(item)
                    
                    normalized['reasoning_content'] = deduplicated
                else:
                    # For other fields, use extracted value
                    normalized[key] = value
                    
        except json.JSONDecodeError as e:
            extraction_notes.append(f"Failed to parse JSON in '{field_name}': {str(e)[:50]}")
            continue
    
    # Step 3: Add processing metadata
    if extraction_notes:
        normalized['_processing_notes'] = '; '.join(extraction_notes)
        normalized['_processed_at'] = datetime.utcnow().isoformat() + 'Z'
    
    # Step 4: Build markdown content
    def build_markdown_sections():
        sections = []
        
        # Reasoning content first
        reasoning = normalized.get('reasoning_content', [])
        if isinstance(reasoning, list):
            reasoning_text = '\n'.join(str(item) for item in reasoning if str(item).strip())
        elif isinstance(reasoning, str):
            reasoning_text = reasoning.strip()
        else:
            reasoning_text = '(No reasoning content found)'
        
        sections.append("# reasoning content\n")
        sections.append("```text\n")
        sections.append(reasoning_text)
        sections.append("\n```\n\n")
        
        # Structured sections in priority order
        priority_fields = ['summary', 'occurrences_summary', 'limitations', 'total_contexts']
        processed_fields = {'reasoning_content', '_processing_notes', '_processed_at'}
        
        for field in priority_fields:
            if field in normalized and field not in processed_fields:
                value = normalized[field]
                sections.append(f"## {field}\n\n")
                if isinstance(value, (str, int, float)):
                    sections.append(f"{value}\n\n")
                else:
                    sections.append(f"{json.dumps(value, indent=2)}\n\n")
                processed_fields.add(field)
        
        # Any remaining fields
        for field, value in normalized.items():
            if field in processed_fields:
                continue
            if isinstance(value, str) and value.strip():
                sections.append(f"## {field}\n\n{value.strip()}\n\n")
            elif isinstance(value, (int, float)):
                sections.append(f"## {field}\n\n{value}\n\n")
        
        # Processing metadata footer
        if extraction_notes:
            sections.append("---\n\n")
            sections.append("### Processing Notes\n\n")
            for note in extraction_notes:
                sections.append(f"- {note}\n")
            sections.append(f"\n_Generated: {datetime.utcnow().isoformat()}Z_\n")
        
        return ''.join(sections)
    
    # Step 5: Write markdown file
    safe_keyword = _sanitize_filename(keyword)
    output_filename = filename or f"{safe_keyword}_filtered_normalized.md"
    
    markdown_content = build_markdown_sections()
    
    with open(output_filename, 'w', encoding='utf-8') as f:
        f.write(markdown_content)
    
    # Step 6: Report results
    chars_written = len(markdown_content)
    extracted_count = len(extraction_notes)
    
    print(f"✅ Filtered analysis exported: {output_filename}")
    print(f"   📄 Content: {chars_written} characters")
    print(f"   🔧 Extractions: {extracted_count} JSON blocks processed")
    if extracted_count > 0:
        print(f"   📋 Notes: {'; '.join(extraction_notes)}")
    
    return output_filename

# Example using the provided structure (assign to variable `blocks_example` before calling if not already)
try:
    if 'blocks_example' in globals():
        export_blocks_reasoning_first(blocks_example, 'textualism')
except Exception as e:
    print('Export failed:', e)


Markdown report written: ordinary_meaning_reasoning_first.md (length=3073 chars)


### Step 1. Read SCOTUS DIY Corpus

- saved as `loc_gov.json`

In [3]:
# read pdf scotus files
df = pd.read_json("../loc_gov.json", lines=True)

df['key'] = df['filename'].apply(lambda x: x.split('usrep')[1][:3])
df['subkey'] = df['filename'].apply(lambda x: x.split('usrep')[1].split('.pdf')[0])

# Create a dictionary to hold the DataFrame contents
df_dict = {}

for _, row in df.iterrows():
    if row['key'] not in df_dict:
        df_dict[row['key']] = {}
    df_dict[row['key']][row['subkey']] = row['content']

# format scotus data for getout_of_text_3, similar to COCA keyword results
db_dict_formatted = {}
for volume, cases in df_dict.items():
    # Create a DataFrame for each volume with case text
    case_data = []
    for case_id, case_text in cases.items():
        case_data.append({'case_id': case_id, 'text': case_text})
    db_dict_formatted[volume] = pd.DataFrame(case_data)


### Step 2. Load Langchain AWS Bedrock Model

Here we are using AWS Bedrock model `openai.gpt-oss-120b-1:0` as the max token count of 128,000 allows for a large context window at a cost-effective price structure. Notably as well we always aim for open models to promote transparent and responsible AI. For more info, see:
- https://openai.com/open-models/
- https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-openai.html

#### Tools

- `search_tool` takes a keyword and analysis type, which allows for a quick summary of a keyword across SCOTUS
- `filtered_tool` takes a filtered corpus dict, allowing for the researcher to control exactly what is passed to AI 

In [4]:
model = init_chat_model(
    "openai.gpt-oss-120b-1:0",
    model_provider="bedrock_converse",
    credentials_profile_name="atn-developer",
    max_tokens=128000
)
# Assume you built db_dict_formatted (volume -> DataFrame with columns ['case_id','text'])
search_tool = ScotusAnalysisTool(model=model, db_dict_formatted=db_dict_formatted)
filtered_tool = ScotusFilteredAnalysisTool(model=model)

### Step 3. Using the `search_tool` for a keyword of interest, namely `bank`

- this is a classic NLP keyword to benchmark results (i.e. financial bank vs riverbank)

In [14]:
keyword="Chevron"

In [15]:
text_result = search_tool._run(keyword=keyword, 
                               analysis_focus="general")
text_result


[{'type': 'reasoning_content',
  'reasoning_content': {'text': 'We need to produce insights on temporal evolution, contextual variation, notable intra-dataset patterns, interpretive themes relevant to ordinary meaning. Use only provided data. No external knowledge. Summarize occurrences by volume/year perhaps? The data includes references to cases spanning 1960s to 2000s. Need to analyze.\n\nWe have many entries with "Chevron Oil Co.", "Chevron U.S.A. Inc.", "Chevron Corp.", etc. Patterns: early references (1970s) about Chevron Oil Co. v. Huson (1971) and retroactivity; later references (1980s-1990s) about Chevron U.S.A. Inc. v. NRDC (1984); later 2000s about Chevron deference and agency interpretation.\n\nTemporal evolution: early 1960s-70s references relate to Chevron Oil Co. (oil industry) and retroactive application; mid-1980s adoption of Chevron doctrine for agency deference; 1990s-2000s proliferation across diverse statutory contexts (EPA, EEOC, labor, tax, securities, etc.) show

In [16]:
export_blocks_reasoning_first(text_result, keyword)

Markdown (reasoning first) written: Chevron_reasoning_first_stream.md | length=11676 chars


'Chevron_reasoning_first_stream.md'

### Step 4. Using the `filtered_tool` for a keyword of interest, namely `dictionary`

- to see how SCOTUS references dictionaries

In [61]:
# After you have filtered JSON:
filtered_json = got3.search_keyword_corpus(keyword, 
                                           db_dict_formatted, 
                                           output="json",
                                           context_words=60,
                                           parallel=True)

analysis = filtered_tool._run(
    keyword=keyword,
    results_json=filtered_json,
    extraction_strategy="all",
    #max_contexts=60, # to filter results
    return_json=True,
    debug=True
)

[DEBUG] raw_chars=192574 extracted_chars=186980 reduction_ratio=0.971 raw≈48144tok extracted≈46745tok strategy=all limit=None


In [62]:
analysis

{'keyword': 'Chevron',
 'total_contexts': 256,
 'occurrences_summary': '256 context snippet(s) across 256 case(s)',
 'reasoning_content': ['Model did not return valid JSON; wrapped raw text.',
  'Multiple contexts allow limited comparative analysis.'],
 'summary': '{\'type\': \'reasoning_content\', \'reasoning_content\': {\'text\': \'We need produce JSON with fields. Use keyword "Chevron". total_contexts = number of contexts which is total cases =256? Actually there are 256 contexts. So total_contexts =256. occurrences_summary: from prompt: "Occurrences Per Case (sample): 353:11=1; 390:17=1; ..." that\\\'s many but sample. We need a summary string describing frequency. Probably note that "Chevron appears in all 256 contexts, with at least one occurrence per case." Use that. reasoning_content: 3-6 steps of analysis: e.g., identify contexts, extract uses, categorize, assess patterns, summarize. summary: overall overview of usage patterns: Chevron mainly appears in case titles referencing

In [63]:
# export to report
normalize_and_export_filtered_analysis(analysis, keyword)

✅ Filtered analysis exported: Chevron_filtered_normalized.md
   📄 Content: 2007 characters
   🔧 Extractions: 1 JSON blocks processed
   📋 Notes: Extracted JSON from 'summary'


'Chevron_filtered_normalized.md'